In [1]:
from pathlib import Path
import sys

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

ROOT = Path("..").resolve()
sys.path.append(str(ROOT / "src"))

from flyby_anomaly import (
    load_flyby_cases_csv,
    compute_classical_delta_v,
    compute_anderson_delta_v,
    compute_busft_diagnostics,
    FlybyCase,
)
from busft_core._kernel import (
    G,
    M_EARTH,
    R_EARTH,
)

CASES_PATH = ROOT / "data" / "flyby_cases.csv"

cases = load_flyby_cases_csv(CASES_PATH)
len(cases)


5

In [2]:
cases = load_flyby_cases_csv(str(ROOT / "data" / "flyby_cases.csv"))
len(cases), cases[0]


(5,
 FlybyCase(mission='Galileo_1', v_inf_km_s=8.949, v_f_km_s=13.738, impact_parameter_km=11261.0, perigee_alt_km=956.053, eccentricity=2.4736, deflection_deg=47.67, inclination_deg=142.9, time_utc='20:34:34', date_utc='1990-12-08', mass_kg=2497.1, alpha_deg=163.7, delta_deg=2.975, delta_v_obs_mm_s=3.92, delta_v_obs_err_mm_s=0.08, delta_v_perigee_obs_mm_s=2.56, delta_v_perigee_obs_err_mm_s=0.05, delta_E_J_per_kg=35.1, delta_E_err_J_per_kg=0.7, delta_in_deg=-12.52, delta_out_deg=-34.15, notes='GEGA1; Anderson 2007 Table 2; δ_in/out from Jouannic 2015 (GL-I)'))

In [3]:
rows = []

for case in cases:
    # klasyczny model: idealny Kepler → Δv_classical = 0
    dv_classical_mm_s = compute_classical_delta_v(case)

    # równanie Andersona (empiryczne)
    dv_anderson_mm_s = compute_anderson_delta_v(case)

    # BUSFT: diagnostyka w perygeum z pełnego kernela
    diag = compute_busft_diagnostics(case)

    rows.append({
        "mission": case.mission,
        "perigee_alt_km": case.perigee_alt_km,
        "v_inf_km_s": case.v_inf_km_s,
        "delta_v_obs_mm_s": case.delta_v_obs_mm_s,
        "delta_v_classical_mm_s": dv_classical_mm_s,
        "delta_v_anderson_mm_s": dv_anderson_mm_s,
        "phi_perigee_J_per_kg": diag.phi_perigee,
        "r_perigee_m": diag.r_perigee,
        "v_inf_m_s": diag.v_inf_m_s,
        "v_perigee_m_s": diag.v_perigee_m_s,
        "B_in": diag.B_in,
        "B_out": diag.B_out,
        "delta_B": diag.delta_B,
        "gamma_in": diag.gamma_in,
        "gamma_out": diag.gamma_out,
        "delta_v_busft_mm_s": diag.delta_v_busft_mm_s,
    })

df_diag = pd.DataFrame(rows)
df_diag


,mission,perigee_alt_km,v_inf_km_s,delta_v_obs_mm_s,delta_v_classical_mm_s,delta_v_anderson_mm_s,phi_perigee_J_per_kg,r_perigee_m,v_inf_m_s,v_perigee_m_s,B_in,B_out,delta_B,gamma_in,gamma_out,delta_v_busft_mm_s
0,Galileo_1,956.053,8.949,3.92,0.0,4.111699,-5.439966e+07,7327053.0,8949.0,13738.0,0.700066,0.700066,-1.626499e-11,1.303866,1.303866,81675.290062
1,NEAR,532.485,6.851,13.46,0.0,13.243100,-5.773739e+07,6903485.0,6851.0,12739.0,0.700063,0.700063,6.900924e-12,1.303865,1.303865,-45265.227204
2,Cassini_1,1171.505,16.010,NaN,0.0,-1.065250,-5.284573e+07,7542505.0,16010.0,19030.0,0.700068,0.700068,-3.360434e-11,1.303866,1.303866,94322.535851
3,Rosetta_1,1954.303,3.863,1.82,0.0,2.060872,-4.787684e+07,8325303.0,3863.0,10517.0,0.700073,0.700073,2.095168e-11,1.303869,1.303869,-243728.128988
4,Messenger,2336.059,4.056,NaN,0.0,0.055157,-4.577771e+07,8707059.0,4056.0,10389.0,0.700076,0.700076,1.864975e-11,1.303870,1.303870,-206626.700785


In [4]:
df_diag[[
    "mission",
    "perigee_alt_km",
    "v_inf_km_s",
    "delta_v_obs_mm_s",
    "B_in",
    "B_out",
    "delta_B",
    "gamma_in",
    "gamma_out",
    "delta_v_busft_mm_s",
]]


,mission,perigee_alt_km,v_inf_km_s,delta_v_obs_mm_s,B_in,B_out,delta_B,gamma_in,gamma_out,delta_v_busft_mm_s
0,Galileo_1,956.053,8.949,3.92,0.700066,0.700066,-1.626499e-11,1.303866,1.303866,81675.290062
1,NEAR,532.485,6.851,13.46,0.700063,0.700063,6.900924e-12,1.303865,1.303865,-45265.227204
2,Cassini_1,1171.505,16.010,NaN,0.700068,0.700068,-3.360434e-11,1.303866,1.303866,94322.535851
3,Rosetta_1,1954.303,3.863,1.82,0.700073,0.700073,2.095168e-11,1.303869,1.303869,-243728.128988
4,Messenger,2336.059,4.056,NaN,0.700076,0.700076,1.864975e-11,1.303870,1.303870,-206626.700785


In [ ]:
import matplotlib.pyplot as plt

# robimy kopię z aliasami, żeby mieć B i gamma_B jak w opisie
df_plot = df_diag.copy()
df_plot["B"] = df_plot["B_out"]          # np. bierzemy B_out jako reprezentatywne
df_plot["gamma_B"] = df_plot["gamma_out"]

display(df_plot[[
    "mission",
    "perigee_alt_km",
    "v_inf_km_s",
    "delta_v_obs_mm_s",
    "delta_v_anderson_mm_s",
    "delta_v_busft_mm_s",
    "B_in", "B_out", "delta_B",
    "gamma_in", "gamma_out",
]])

# --- Wykres B ---
plt.figure()
plt.plot(df_plot["mission"], df_plot["B"], marker="o")
plt.axhline(0.0, linestyle="--")
plt.ylabel("B (dimensionless)")
plt.title("BUSFT B correction at perigee for flyby cases")
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()
plt.show()

# --- Wykres gamma_B ---
plt.figure()
plt.plot(df_plot["mission"], df_plot["gamma_B"], marker="o")
plt.ylabel("gamma_B")
plt.title("BUSFT gamma factor at perigee for flyby cases")
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
df_model = df_diag.copy()

# Zestawienie obserwacji, Andersona i BUSFT (Δv_BUSFT wyprowadzone bezpośrednio z równania Baranowicza)
df_model[[
    "mission",
    "delta_v_obs_mm_s",
    "delta_v_anderson_mm_s",
    "delta_v_busft_mm_s",
]]


In [ ]:
plt.figure()
x = np.arange(len(df_model))

obs = df_model["delta_v_obs_mm_s"].values
model = df_model["delta_v_busft_mm_s"].values

width = 0.35

plt.bar(x - width/2, obs, width=width, label="Observed Δv [mm/s]")
plt.bar(x + width/2, model, width=width, label="BUSFT model Δv [mm/s]")

plt.xticks(x, df_model["mission"], rotation=45)
plt.ylabel("Δv [mm/s]")
plt.title("Earth flyby anomaly: observed vs BUSFT-model prediction (hypothesis)")
plt.grid(True, axis="y")
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
# Porównanie dopasowania: Anderson vs BUSFT
mask_obs = df_model["delta_v_obs_mm_s"].notna()

obs = df_model.loc[mask_obs, "delta_v_obs_mm_s"].values
pred_anderson = df_model.loc[mask_obs, "delta_v_anderson_mm_s"].values
pred_busft = df_model.loc[mask_obs, "delta_v_busft_mm_s"].values

def r2_score(y_true, y_pred):
    if len(y_true) < 2:
        return np.nan
    ss_res = np.sum((y_true - y_pred)**2)
    ss_tot = np.sum((y_true - np.mean(y_true))**2)
    return 1.0 - ss_res/ss_tot if ss_tot != 0 else np.nan

r2_anderson = r2_score(obs, pred_anderson)
r2_busft = r2_score(obs, pred_busft)

r2_anderson, r2_busft
